In [ ]:
import sagemaker
import os
from datetime import datetime
from sagemaker.utils import sagemaker_timestamp
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### SETUP

In [ ]:
# environment setup
ROLE = get_execution_role()
SESSION = sagemaker.Session()
BUCKET = 's3://sagemaker-training-itc309'
TIMESTAMP = datetime.now().strftime("%d-%m-%Y----%H-%M-%S")

# instance setup
# INSTANCE_TYPE = 'ml.m5.large' # $0.115 ph / 8gb ram / 2 vCPU  (More ram needed)
INSTANCE_TYPE = 'ml.m5.2xlarge' # $0.461 ph / 32gb ram / 8 vCPU 
INSTANCE_COUNT = 1

# path to training script
TRAINING_SCRIPT = "train.py"

# name of the job (will be used as folder name in s3)
JOB_NAME = 'training'


### HYPERPARAMETERS

In [ ]:
hyperparameters = {
    "epochs": 2,
    "bucket": BUCKET,
    # "retrain-model": True,
}

### TRAIN

In [ ]:
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
estimator = TensorFlow(
    role=ROLE,
    entry_point=TRAINING_SCRIPT,
    output_path=f'{BUCKET}/training_output',
    instance_count=INSTANCE_COUNT,
    instance_type=INSTANCE_TYPE,
    hyperparameters=hyperparameters,
    py_version='py37',
    framework_version = '2.3',
    source_dir='src'
)

response = estimator.fit({"train": BUCKET}, job_name=f'{JOB_NAME}----{TIMESTAMP}')

print(response)